# Prepare

**Remeber Goal of project**
- Find drivers of collisions and how to reduce death

**import**

In [1]:
# For funtion annotations
from binascii import a2b_qp
from typing import Union
from typing import Tuple

# data manipulation
import pandas as pd
import numpy as np

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# data separation/transformation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# system manipulation
import os
import sys
sys.path.append("./util_")
import acquire_
import prepare_

# other
import env
import warnings
warnings.filterwarnings("ignore")


### get data

In [2]:
# get data from acquire file
vehicle = acquire_.get_data()
vehicle.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,09/11/2021,9:35,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN


## Rename column names

In [3]:
# make the columsn lower case
# replace spaces
vehicle.columns = vehicle.columns.str.lower().str.strip().str.replace("†","").str.replace(" ","_")
vehicle.head(1)

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,09/11/2021,2:39,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN


## Fix column data types

**Object to datetime**

In [4]:
# convert string to datetime
vehicle["crash_date"] = pd.to_datetime(vehicle['crash_date'])
vehicle["crash_time"] = pd.to_datetime(vehicle['crash_time'])

In [5]:
vehicle

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2021-09-11,2023-08-06 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26,2023-08-06 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29,2023-08-06 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11,2023-08-06 09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14,2023-08-06 08:13:00,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014464,2023-07-03,2023-08-06 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2014465,2023-07-22,2023-08-06 21:39:00,BRONX,10457,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2014466,2023-07-02,2023-08-06 17:55:00,MANHATTAN,10006,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2014467,2023-07-22,2023-08-06 13:15:00,QUEENS,11433,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


**drop nulls**

In [6]:
# count of nulls in each column
vehicle.isna().sum()

crash_date                             0
crash_time                             0
borough                           626666
zip_code                          626904
latitude                          229416
longitude                         229416
location                          229416
on_street_name                    423681
cross_street_name                 753516
off_street_name                  1683160
number_of_persons_injured             18
number_of_persons_killed              31
number_of_pedestrians_injured          0
number_of_pedestrians_killed           0
number_of_cyclist_injured              0
number_of_cyclist_killed               0
number_of_motorist_injured             0
number_of_motorist_killed              0
contributing_factor_vehicle_1       6328
contributing_factor_vehicle_2     307078
contributing_factor_vehicle_3    1871702
contributing_factor_vehicle_4    1982443
contributing_factor_vehicle_5    2005827
collision_id                           0
vehicle_type_cod

In [7]:
# remove every location data that contains nulls
vehicle = vehicle[vehicle.latitude.notna()]
vehicle = vehicle[vehicle.longitude.notna()]
vehicle = vehicle[vehicle.zip_code.notna()]

# removeing all the null columns for number_of_persons_injured and number_of_persons_killed
vehicle = vehicle[vehicle.number_of_persons_injured.notna()]
vehicle = vehicle[vehicle.number_of_persons_killed.notna()]

vehicle.shape

(1351308, 29)

**Float to intiger**

In [8]:
# change column data type from float to int
vehicle.number_of_persons_injured = vehicle.number_of_persons_injured.astype("int")
vehicle.number_of_persons_killed = vehicle.number_of_persons_killed.astype("int")

**Fill nulls**

In [9]:
# count number of nulls remailing in numeric columns
vehicle.select_dtypes("number").isna().sum()

latitude                         0
longitude                        0
number_of_persons_injured        0
number_of_persons_killed         0
number_of_pedestrians_injured    0
number_of_pedestrians_killed     0
number_of_cyclist_injured        0
number_of_cyclist_killed         0
number_of_motorist_injured       0
number_of_motorist_killed        0
collision_id                     0
dtype: int64

In [10]:
# fill nulls with Unkown for all object column types
vehicle.fillna("UNKNOWN", inplace=True)
vehicle.head()

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
3,2021-09-11,2023-08-06 09:35:00,BROOKLYN,11208,40.667202,-73.866500,"(40.667202, -73.8665)",UNKNOWN,UNKNOWN,1211 LORING AVENUE,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,4456314,Sedan,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
4,2021-12-14,2023-08-06 08:13:00,BROOKLYN,11233,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,UNKNOWN,...,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,4486609,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
7,2021-12-14,2023-08-06 08:17:00,BRONX,10475,40.868160,-73.831480,"(40.86816, -73.83148)",UNKNOWN,UNKNOWN,344 BAYCHESTER AVENUE,...,Unspecified,UNKNOWN,UNKNOWN,UNKNOWN,4486660,Sedan,Sedan,UNKNOWN,UNKNOWN,UNKNOWN
8,2021-12-14,2023-08-06 21:10:00,BROOKLYN,11207,40.671720,-73.897100,"(40.67172, -73.8971)",UNKNOWN,UNKNOWN,2047 PITKIN AVENUE,...,Unspecified,UNKNOWN,UNKNOWN,UNKNOWN,4487074,Sedan,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN
9,2021-12-14,2023-08-06 14:58:00,MANHATTAN,10017,40.751440,-73.973970,"(40.75144, -73.97397)",3 AVENUE,EAST 43 STREET,UNKNOWN,...,Unspecified,UNKNOWN,UNKNOWN,UNKNOWN,4486519,Sedan,Station Wagon/Sport Utility Vehicle,UNKNOWN,UNKNOWN,UNKNOWN


## Group alike objects

In [11]:
# unique counts in object columns
vehicle.select_dtypes("object").nunique()

borough                               5
zip_code                            233
location                         245220
on_street_name                    11098
cross_street_name                 13338
off_street_name                  190463
contributing_factor_vehicle_1        62
contributing_factor_vehicle_2        61
contributing_factor_vehicle_3        49
contributing_factor_vehicle_4        35
contributing_factor_vehicle_5        25
vehicle_type_code_1                1188
vehicle_type_code_2                1333
vehicle_type_code_3                 177
vehicle_type_code_4                  75
vehicle_type_code_5                  57
dtype: int64

**Contributing factor**

In [12]:
# Group similar factors together   
contributing_factor = {
    "Visibility and Road Conditions":["Windshield Inadequate",
                                      "Headlights Defective",
                                      "Other Lighting Defects",
                                      "Glare",
                                      "View Obstructed/Limited",
                                      "Pavement Slippery",
                                      "Obstruction/Debris",
                                      "Pavement Defective"],
    "Distractions from Electronic Devices":["Cell Phone (hand-Held)", "Cell Phone (hand-held)",
                                            "Cell Phone (hands-free)",
                                            "Other Electronic Device",
                                            "Outside Car Distraction"],
    "Impairment (Alcohol, Drugs, Medication)":["Alcohol Involvement",
                                               "Drugs (illegal)","Drugs (Illegal)",
                                               "Prescription Medication"],
    "Driver Fatigue and Inattention":["Fell Asleep",
                                      "Lost Consciousness",
                                      "Fatigued/Drowsy",
                                      "Illnes", "Illness",
                                      "Unsafe Speed",
                                      "Driver Inattention/Distraction",
                                     "80"],
    "Unsafe Driving Maneuvers":["Unsafe Lane Changing",
                                "Passing Too Closely",
                                "Turning Improperly",
                                "Passing or Lane Usage Improper",
                                "Failure to Yield Right-of-Way",
                                "Failure to Keep Right"],
    "Vehicle Equipment Failure":["Following Too Closely",
                                 "Traffic Control Disregarded",
                                 "Accelerator Defective",
                                 "Brakes Defective",
                                 "Steering Failure",
                                 "Tire Failure/Inadequate"],
    "Issues with Traffic Control and Lane Marking":["Traffic Control Device Improper/Non-Working",
                                                    "Lane Marking Improper/Inadequate"],
    "Driver Characteristics and Experience":["Physical Disability",
                                             "Driver Inexperience"],
    "Reactions to Other Vehicles":["Reaction to Other Uninvolved Vehicle",
                                   "Reaction to Uninvolved Vehicle"],
    "Distracted Driving":["Listening/Using Headphones",
                          "Texting",
                          "Eating or Drinking",
                         "Distracted Driving"],
    "Vehicle-related Incidents":["Vehicle Vandalism",
                                 "Tow Hitch Defective",
                                 "Driverless/Runaway Vehicle",
                                 "Oversized Vehicle",
                                 "Other Vehicular"],
    "Interactions with Pedestrians and Cyclists":["Animals Action",
                                                  "Pedestrian/Bicyclist/Other Pedestrian Error/Confusion"],
    "Aggressive Driving and Passenger Distraction":["Aggressive Driving/Road Rage",
                                                    "Passenger Distraction"],
    "Unsafe Lane Changes and Backing":["Unsafe Lane Changing",
                                       "Passing Too Closely",
                                       "Turning Improperly",
                                       "Passing or Lane Usage Improper",
                                       "Backing Unsafely"],
    "Other":["Using On Board Navigation Device",
            "Tinted Windows",
            "Shoulders Defective/Improper"],
    "Unspecified Factors":["UNKNOWN",
                                        "Unspecified",
                                       "1",
                                       ""]
}


# Replacing values in the "vehicle_type_code_1" column based on the categories
def replace_category_factor(value):
    for category, codes in contributing_factor.items():
        if value in codes:
            return category
    return 'OTHER'

# apply the created function to the data
vehicle["contributing_factor_vehicle_1"] = vehicle["contributing_factor_vehicle_1"].apply(replace_category_factor)
vehicle["contributing_factor_vehicle_2"] = vehicle["contributing_factor_vehicle_2"].apply(replace_category_factor)
vehicle["contributing_factor_vehicle_3"] = vehicle["contributing_factor_vehicle_3"].apply(replace_category_factor)
vehicle["contributing_factor_vehicle_4"] = vehicle["contributing_factor_vehicle_4"].apply(replace_category_factor)
vehicle["contributing_factor_vehicle_5"] = vehicle["contributing_factor_vehicle_5"].apply(replace_category_factor)      


In [13]:
contributing_factor_cate = {
        'unspecified': ['Unspecified Factors'],
        'driver': ['Driver Fatigue and Inattention', 'Aggressive Driving and Passenger Distraction'],
        'unsafe driving': ['Unsafe Driving Maneuvers', 'Unsafe Lane Changes and Backing'],
        'vehicle failure': ['Vehicle Equipment Failure', 'Vehicle-related Incidents'],
        'visibility': ['Visibility and Road Conditions', 'Issues with Traffic Control and Lane Marking'],
        'driver experience': ['Driver Characteristics and Experience'],
        'impairment': ['Impairment (Alcohol, Drugs, Medication)'],
        'reactions': ['Reactions to Other Vehicles'],
        'distractions': ['Distractions from Electronic Devices', 'Interactions with Pedestrians and Cyclists', 'Distracted Driving'],
        'other': ['Other']
    }

# Replacing values in the "vehicle_type_code_1" column based on the categories
def replace_contributing_category(value):
    for category, codes in contributing_factor_cate.items():
        if value in codes:
            return category
    return 'OTHER'

# apply the created function to the data
vehicle["contributing_factor_vehicle_1"] = vehicle["contributing_factor_vehicle_1"].apply(replace_contributing_category)
vehicle["contributing_factor_vehicle_2"] = vehicle["contributing_factor_vehicle_2"].apply(replace_contributing_category)
vehicle["contributing_factor_vehicle_3"] = vehicle["contributing_factor_vehicle_3"].apply(replace_contributing_category)
vehicle["contributing_factor_vehicle_4"] = vehicle["contributing_factor_vehicle_4"].apply(replace_contributing_category)
vehicle["contributing_factor_vehicle_5"] = vehicle["contributing_factor_vehicle_5"].apply(replace_contributing_category)

**Vehicle type**

In [14]:
# Separete large vehicle control code from smaller counts
veh_contr_code = vehicle.vehicle_type_code_1.value_counts()[vehicle.vehicle_type_code_1.value_counts() > 20]

print("Count of categories greater than 20:",len(veh_contr_code))
veh_contr_code

Count of categories greater than 20: 107


Sedan                                  342856
PASSENGER VEHICLE                      310338
Station Wagon/Sport Utility Vehicle    263726
SPORT UTILITY / STATION WAGON          134198
Taxi                                    31193
                                        ...  
Trail                                      22
PEDICAB                                    22
FDNY AMBUL                                 22
BOX T                                      22
FIRETRUCK                                  21
Name: vehicle_type_code_1, Length: 107, dtype: int64

In [15]:
# Group the categories
vehicle_categories = {
    'SEDAN': ['Sedan', '4 dr sedan', '2 dr sedan', '3-Door'],
    'STATION_WAGON': ['Station Wagon/Sport Utility Vehicle', 'SPORT UTILITY / STATION WAGON'],
    'PASSENGER_VEHICLE': ['PASSENGER VEHICLE'],
    'TAXI': ['Taxi', 'TAXI'],
    'PICKUP_TRUCK': ['Pick-up Truck', 'PICK-UP TRUCK', 'PK', 'Pickup with mounted Camper', 'pick'],
    'UNKNOWN': ['UNKNOWN', 'UNKNO', 'UNK','unknown', 'unko', 'unk'],
    'VAN': ['VAN', 'van', 'Van', 'Van Camper'],
    'BOX_TRUCK': ['Box Truck', 'BOX T', 'BOX TRUCK'],
    'BUS': ['Bus', 'BUS', 'School Bus'],
    'LARGE_COM_VEH': ['LARGE COM VEH(6 OR MORE TIRES)'],
    'SMALL_COM_VEH': ['SMALL COM VEH(4 TIRES)', 'COMME'],
    'LIVERY_VEHICLE': ['LIVERY VEHICLE'],
    'TRACTOR_TRUCK_DIESEL': ['Tractor Truck Diesel', 'TRACT'],
    'MOTORCYCLE': ['Bike', 'MOTORCYCLE', 'Motorscooter', 'Moped', 'Minibike', 'Minicycle'],
    'AMBULANCE': ['Ambulance', 'AMBULANCE', 'AMBUL', 'ambul', 'AMB', 'FDNY AMBUL', 'fdny'],
    'CONVERTIBLE': ['Convertible'],
    'DUMP': ['Dump', 'dump'],
    'E_BIKE': ['E-Bike', 'E-Bik', 'ELECT'],
    'FLAT_BED': ['Flat Bed', 'FLAT'],
    'GARBAGE_OR_REFUSE': ['Garbage or Refuse'],
    'CARRY_ALL': ['Carry All'],
    'E_SCOOTER': ['E-Scooter', 'E-Sco'],
    'TRACTOR_TRUCK_GASOLINE': ['Tractor Truck Gasoline', 'FORD'],
    'TOW_TRUCK': ['Tow Truck / Wrecker', 'Tow Truck', 'TOW T'],
    'FIRE_TRUCK': ['FIRE TRUCK', 'FIRET', 'Fire Truck', 'fire', 'FDNY FIRE', 'FDNY TRUCK', 'FDNY'],
    'CHASSIS_CAB': ['Chassis Cab'],
    'TANKER': ['Tanker', 'TANK'],
    'REFRIGERATED_VAN': ['Refrigerated Van'],
    'CONCRETE_MIXER': ['Concrete Mixer'],
    'FLAT_RACK': ['Flat Rack'],
    'ARMORED_TRUCK': ['Armored Truck'],
    'BEVERAGE_TRUCK': ['Beverage Truck'],
    'SCOOTER': ['SCOOTER', 'SCOOT'],
    'LIMO': ['LIMO'],
    'LIFT_BOOM': ['Lift Boom'],
    'TRUCK': ['TRUCK', 'truck'],
    'TRAILER': ['TRAIL', 'trail', 'TRAILER'],
    'STAKE_OR_RACK': ['Stake or Rack'],
    'LUNCH_WAGON': ['Lunch Wagon'],
    'FORKLIFT': ['FORKL'],
    'MOTORIZED_HOME': ['Motorized Home'],
    'PEDICAB': ['Pedicab'],
    'HOPPER': ['Hopper'],
    'MULTI_WHEELED_VEHICLE': ['Multi-Wheeled Vehicle'],
    'USPS': ['USPS'],
    'DELIVERY': ['DELIV', 'DELV'],
    'UTILITY': ['UTILI', 'UTIL'],
    'OPEN_BODY': ['Open Body'],
    'BULK_AGRICULTURE': ['Bulk Agriculture']
}
    

# Replacing values in the "vehicle_type_code_1" column based on the categories
def replace_category(value):
    for category, codes in vehicle_categories.items():
        if value in codes:
            return category
    return 'OTHER'

# apply the created function to the data
vehicle["vehicle_type_code_1"] = vehicle["vehicle_type_code_1"].apply(replace_category)
vehicle["vehicle_type_code_2"] = vehicle["vehicle_type_code_2"].apply(replace_category)
vehicle["vehicle_type_code_3"] = vehicle["vehicle_type_code_3"].apply(replace_category)
vehicle["vehicle_type_code_4"] = vehicle["vehicle_type_code_4"].apply(replace_category)
vehicle["vehicle_type_code_5"] = vehicle["vehicle_type_code_5"].apply(replace_category)


In [16]:
# covertinto even smaller cathegories
vehicle_type = {
        'car': ['SEDAN', 'PASSENGER_VEHICLE', 'TAXI', 'CONVERTIBLE', 'LIMO'],
        'truck': ['PICKUP_TRUCK', 'BOX_TRUCK', 'LARGE_COM_VEH', 'TRACTOR_TRUCK_DIESEL', 'TRACTOR_TRUCK_GASOLINE',
                  'TOW_TRUCK', 'FIRE_TRUCK', 'BEVERAGE_TRUCK', 'REFRIGERATED_VAN', 'CONCRETE_MIXER', 'ARMORED_TRUCK',
                  'DELIVERY', 'BULK_AGRICULTURE', 'SMALL_COM_VEH', 'MOTORIZED_HOME'],
        'van': ['STATION_WAGON', 'VAN'],
        'bus': ['BUS'],
        'motorcycle': ['MOTORCYCLE'],
        'scooter': ['E_BIKE', 'E_SCOOTER', 'SCOOTER'],
        'trailer': ['TRAILER'],
        'other': ['OTHER', 'USPS', 'PEDICAB', 'FORKLIFT', 'LUNCH_WAGON', 'HOPPER'],
        'unknown': ["UNKNOWN"]
    }

# Replacing values in the "vehicle_type_code_1" column based on the categories
def replace_category_type(value):
    for category, codes in vehicle_type.items():
        if value in codes:
            return category
    return 'OTHER'
# apply the created function to the data
vehicle["vehicle_type_code_1"] = vehicle["vehicle_type_code_1"].apply(replace_category_type)
vehicle["vehicle_type_code_2"] = vehicle["vehicle_type_code_2"].apply(replace_category_type)
vehicle["vehicle_type_code_3"] = vehicle["vehicle_type_code_3"].apply(replace_category_type)
vehicle["vehicle_type_code_4"] = vehicle["vehicle_type_code_4"].apply(replace_category_type)
vehicle["vehicle_type_code_5"] = vehicle["vehicle_type_code_5"].apply(replace_category_type)


In [17]:
# unique counts in object columns
vehicle.select_dtypes("object").nunique()

borough                               5
zip_code                            233
location                         245220
on_street_name                    11098
cross_street_name                 13338
off_street_name                  190463
contributing_factor_vehicle_1        10
contributing_factor_vehicle_2        10
contributing_factor_vehicle_3        10
contributing_factor_vehicle_4         9
contributing_factor_vehicle_5         8
vehicle_type_code_1                  10
vehicle_type_code_2                  10
vehicle_type_code_3                  10
vehicle_type_code_4                  10
vehicle_type_code_5                   9
dtype: int64

## Create my target variable

In [18]:
# if any fatalities are found add 1 in the list
fatalities = []
for row in range(len(vehicle)):
    # locate each row and check if any number of fatalities accured
    if vehicle.number_of_persons_killed.iloc[row] != 0:
        fatalities.append(1)
    elif vehicle.number_of_pedestrians_killed.iloc[row] != 0:
        fatalities.append(1)
    elif vehicle.number_of_cyclist_killed.iloc[row] != 0:
        fatalities.append(1)
    elif vehicle.number_of_motorist_killed.iloc[row] != 0:
        fatalities.append(1)
    else:
        fatalities.append(0)

In [19]:
# add target variable to the data
vehicle["fatality"] = fatalities
vehicle.head(1)

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,cross_street_name,off_street_name,...,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,fatality
3,2021-09-11,2023-08-06 09:35:00,BROOKLYN,11208,40.667202,-73.8665,"(40.667202, -73.8665)",UNKNOWN,UNKNOWN,1211 LORING AVENUE,...,unspecified,unspecified,unspecified,4456314,car,unknown,unknown,unknown,unknown,0


## Remove columns

- remove `location` because it's not tidy and there is also longitude and latitude describing this column.
- remove `collision_id` because it won't help identify my target
- remove `number_of_persons, pedestrians, cyclist, and motorist killed` beacuse they have been moved into the `fatality` variable.

In [20]:
# drop redundent columns
remove_cols = ["collision_id",
               "number_of_persons_killed",
              "number_of_pedestrians_killed",
              "number_of_cyclist_killed",
              "number_of_motorist_killed",
               "location", "on_street_name",
               "cross_street_name", "off_street_name"]
vehicle = prepare_.drop_cols(vehicle, remove_cols)

Original dataframe size: (1351308, 30)
New dataframe size: (1351308, 21)


In [21]:
vehicle.head(2)

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_pedestrians_injured,number_of_cyclist_injured,number_of_motorist_injured,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,fatality
3,2021-09-11,2023-08-06 09:35:00,BROOKLYN,11208,40.667202,-73.866500,0,0,0,0,...,unspecified,unspecified,unspecified,unspecified,car,unknown,unknown,unknown,unknown,0
4,2021-12-14,2023-08-06 08:13:00,BROOKLYN,11233,40.683304,-73.917274,0,0,0,0,...,unspecified,unspecified,unspecified,unspecified,unknown,unknown,unknown,unknown,unknown,0


**lower case cell**

In [22]:
# make every cell lower case
for col in vehicle.select_dtypes("object").columns:
    vehicle[col] = vehicle[col].str.lower()

**Remove Outliers**

In [23]:
# # remove outliers
# vehicle = vehicle[~vehicle.contributing_factor_vehicle_1.str.contains("uncertain", case=False)]
# vehicle = vehicle[~vehicle.vehicle_type_code_1.str.contains("other", case=False)]
# vehicle.shape

In [24]:
# have the current sate of the dataframe to a new variable
# This is creatreate to save in csv later
vehicle_before_encoding = vehicle.copy()
vehicle_before_encoding.head(1)

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_pedestrians_injured,number_of_cyclist_injured,number_of_motorist_injured,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,fatality
3,2021-09-11,2023-08-06 09:35:00,brooklyn,11208,40.667202,-73.8665,0,0,0,0,...,unspecified,unspecified,unspecified,unspecified,car,unknown,unknown,unknown,unknown,0


## Encode categorical variable

In [25]:
# unique counts in object columns
vehicle.select_dtypes("object").nunique()

borough                            5
zip_code                         233
contributing_factor_vehicle_1     10
contributing_factor_vehicle_2     10
contributing_factor_vehicle_3     10
contributing_factor_vehicle_4      9
contributing_factor_vehicle_5      8
vehicle_type_code_1                9
vehicle_type_code_2                9
vehicle_type_code_3                9
vehicle_type_code_4                9
vehicle_type_code_5                8
dtype: int64

In [26]:
# get all columns from dataframe
all_columns = vehicle.columns

# containers of different variable types
categorical = []

# separate variables
for col in all_columns:
    # count number of unique valus in the column
    len_of_uniq = len(vehicle[col].unique())
    
    # also checking for only object data types
    if (col != "fatality") and (len_of_uniq <= 16) and (vehicle[col].dtype == "O"):
        categorical.append(col)
    else: pass

In [27]:
categorical

['borough',
 'contributing_factor_vehicle_1',
 'contributing_factor_vehicle_2',
 'contributing_factor_vehicle_3',
 'contributing_factor_vehicle_4',
 'contributing_factor_vehicle_5',
 'vehicle_type_code_1',
 'vehicle_type_code_2',
 'vehicle_type_code_3',
 'vehicle_type_code_4',
 'vehicle_type_code_5']

In [28]:
# create dummies of the categorical columns
dummies = pd.get_dummies(vehicle[categorical])

dummies.head(3)

,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,contributing_factor_vehicle_1_distractions,contributing_factor_vehicle_1_driver,contributing_factor_vehicle_1_driver experience,contributing_factor_vehicle_1_impairment,contributing_factor_vehicle_1_other,...,vehicle_type_code_4_unknown,vehicle_type_code_4_van,vehicle_type_code_5_bus,vehicle_type_code_5_car,vehicle_type_code_5_motorcycle,vehicle_type_code_5_other,vehicle_type_code_5_scooter,vehicle_type_code_5_truck,vehicle_type_code_5_unknown,vehicle_type_code_5_van
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
7,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [29]:
# renmae the dummie columns
dummies = dummies.rename(columns=lambda x: x.lower().replace("-", "_"))
dummies.head(1)

,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,contributing_factor_vehicle_1_distractions,contributing_factor_vehicle_1_driver,contributing_factor_vehicle_1_driver experience,contributing_factor_vehicle_1_impairment,contributing_factor_vehicle_1_other,...,vehicle_type_code_4_unknown,vehicle_type_code_4_van,vehicle_type_code_5_bus,vehicle_type_code_5_car,vehicle_type_code_5_motorcycle,vehicle_type_code_5_other,vehicle_type_code_5_scooter,vehicle_type_code_5_truck,vehicle_type_code_5_unknown,vehicle_type_code_5_van
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [30]:
# add dummies to the dataset
vehicle_encoded = pd.concat([vehicle, dummies], axis=1)
vehicle_encoded.head(2)

,crash_date,crash_time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_pedestrians_injured,number_of_cyclist_injured,number_of_motorist_injured,...,vehicle_type_code_4_unknown,vehicle_type_code_4_van,vehicle_type_code_5_bus,vehicle_type_code_5_car,vehicle_type_code_5_motorcycle,vehicle_type_code_5_other,vehicle_type_code_5_scooter,vehicle_type_code_5_truck,vehicle_type_code_5_unknown,vehicle_type_code_5_van
3,2021-09-11,2023-08-06 09:35:00,brooklyn,11208,40.667202,-73.866500,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,2021-12-14,2023-08-06 08:13:00,brooklyn,11233,40.683304,-73.917274,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [31]:
vehicle.fatality.value_counts()

0    1349695
1       1613
Name: fatality, dtype: int64

## Split data

In [32]:
# 60/20/20 split
# split test data
train_validate, test = train_test_split(vehicle_encoded, 
                                        test_size=0.2, # 20% in the test set
                                        random_state=95,
                                        stratify = vehicle.fatality)
# split validate data
train, validate = train_test_split(train_validate, 
                                   test_size=0.2/(1-0.2),  # 20% in the validation set
                                    random_state=95,
                                   stratify = train_validate.fatality)

In [33]:
(train.shape, validate.shape,test.shape)

((810784, 117), (270262, 117), (270262, 117))

**Save .csv**

In [34]:
# using the function to same the files
prepare_.save_split_data_(original_df= vehicle_before_encoding,
                          encoded_scaled_df=vehicle_encoded,
                          train= train, validate= validate,
                          test=test, test_size=0.2,
                         stratify_col="fatality",
                         random_state=95)

'SIX data sets saved as .csv'

### Take aways

- Renamed all columns by replacing empty spaces with underscores and making them all lower case
- Remove nulls in location data (zipcode, logitude ...)
- change column data type:
    - object to datetime: `crash_date` and `crash_time`
- fill all other nulls with `UNKNOWN` for object data types
- Group into similar categories to reduce the size of the unique features:
    - `contributing_factor_vehicle` and `vehicle_type_code`
- create target variable as a binary column using:
    - `number_of_persons_killed`, `number_of_pedestrians_killed`, `number_of_cyclist_killed`, and `number_of_motorist_killed`
- encode all cate gorical columns:
    - 299 columns dummies 
    - 299 columns new data columns
 - split data using a 60/20/20